In [142]:
import landmarks
import sklearn.model_selection
import sklearn.metrics
import sklearn.svm
import pandas as pd
import numpy as np
import random

random.seed(0)

# Task A1
Gender detection: male or female.

Import raw data.

In [145]:
a1_X_train_raw, a1_y_train_raw = landmarks.extract_features_labels("celeba")
a1_X_test_raw, a1_y_test_raw = landmarks.extract_features_labels("celeba_test")

Convert raw data into Pandas DataFrames, then split train set into train set and validation set.

In [146]:
# gender == 0 means female
a1_X_labels = np.array([(f"x_{i}", f"y_{i}") for i in range(68)]).flatten()

a1_X_train = pd.DataFrame(data=a1_X_train_raw.reshape(a1_X_train_raw.shape[0], -1), columns=a1_X_labels)
a1_y_train = pd.DataFrame(data=a1_y_train_raw, columns=["image_name", "gender", "smiling"])

a1_X_train, a1_X_validation, a1_y_train, a1_y_validation = sklearn.model_selection.train_test_split(a1_X_train, a1_y_train, train_size=0.75)

a1_X_test = pd.DataFrame(data=a1_X_test_raw.reshape(a1_X_test_raw.shape[0], -1), columns=a1_X_labels)
a1_y_test = pd.DataFrame(data=a1_y_test_raw, columns=["image_name", "gender", "smiling"])

Train and test the model.

In [147]:
a1_model = sklearn.svm.SVC(kernel="linear")

a1_model.fit(a1_X_train, a1_y_train["gender"])

sklearn.metrics.accuracy_score(a1_y_validation["gender"], a1_model.predict(a1_X_validation))

0.9182652210175146

# Task A2
Emotion detection: smiling or not smiling.

Import raw data.

In [149]:
a2_X_train_raw, a2_y_train_raw = landmarks.extract_features_labels("celeba")
a2_X_test_raw, a2_y_test_raw = landmarks.extract_features_labels("celeba_test")

Convert raw data into Pandas DataFrames, then split train set into train set and validation set.

In [150]:
# gender == 0 means female
a2_X_labels = np.array([(f"x_{i}", f"y_{i}") for i in range(68)]).flatten()

a2_X_train = pd.DataFrame(data=a2_X_train_raw.reshape(a2_X_train_raw.shape[0], -1), columns=a2_X_labels)
a2_y_train = pd.DataFrame(data=a2_y_train_raw, columns=["image_name", "gender", "smiling"])

a2_X_train, a2_X_validation, a2_y_train, a2_y_validation = sklearn.model_selection.train_test_split(a2_X_train, a2_y_train, train_size=0.75)

a2_X_test = pd.DataFrame(data=a2_X_test_raw.reshape(a2_X_test_raw.shape[0], -1), columns=a2_X_labels)
a2_y_test = pd.DataFrame(data=a2_y_test_raw, columns=["image_name", "gender", "smiling"])

Train and test the model.

In [151]:
a2_model = sklearn.svm.SVC(kernel="linear")

a2_model.fit(a2_X_train, a2_y_train["gender"])

sklearn.metrics.accuracy_score(a2_y_validation["gender"], a2_model.predict(a2_X_validation))

0.914095079232694

# Task B1
Face shape recognition: 5 types of face shapes

# Task B2
Eye color recognition: 5 types of eye colors

## Print out your results with following format

In [3]:
print(
    "TA1:{},{};TA2:{},{};TB1:{},{};TB2:{},{};".format(
        acc_A1_train,
        acc_A1_test,
        acc_A2_train,
        acc_A2_test,
        acc_B1_train,
        acc_B1_test,
        acc_B2_train,
        acc_B2_test,
    )
)

NameError: name 'acc_A1_train' is not defined